In [35]:
# Libraries
using DataFrames
using CSV
using StatsPlots
using Statistics

using StatsModels
using GLM

In [36]:
# Import data and see first observations
df = CSV.read("../../data/processed_esti.csv", DataFrame)
first(df, 5)

Row,y,w,gender_female,gender_male,gender_transgender,ethnicgrp_asian,ethnicgrp_black,ethnicgrp_mixed_multiple,ethnicgrp_other,ethnicgrp_white,partners1,postlaunch,msm,age,imd_decile
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,1,0,1,0,0,0,1,0,0,0,1,0,27,5
2,0,0,0,1,0,0,0,0,0,1,0,0,0,19,6
3,0,1,0,1,0,0,1,0,0,0,0,1,0,26,4
4,0,0,1,0,0,0,0,0,0,1,1,0,0,20,2
5,1,1,1,0,0,1,0,0,0,0,0,1,0,24,3


In [37]:
describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Int64,Float64,Int64,Int64,DataType
1,y,0.351926,0,0.0,1,0,Int64
2,w,0.529615,0,1.0,1,0,Int64
3,gender_female,0.584244,0,1.0,1,0,Int64
4,gender_male,0.413456,0,0.0,1,0,Int64
5,gender_transgender,0.00230017,0,0.0,1,0,Int64
6,ethnicgrp_asian,0.0638298,0,0.0,1,0,Int64
7,ethnicgrp_black,0.0862565,0,0.0,1,0,Int64
8,ethnicgrp_mixed_multiple,0.0885566,0,0.0,1,0,Int64
9,ethnicgrp_other,0.013226,0,0.0,1,0,Int64


In [38]:
control = select(filter(row -> row[:w] == 0, df), Not(:y))
treatment = select(filter(row -> row[:w] == 1, df), Not(:y))

Row,w,gender_female,gender_male,gender_transgender,ethnicgrp_asian,ethnicgrp_black,ethnicgrp_mixed_multiple,ethnicgrp_other,ethnicgrp_white,partners1,postlaunch,msm,age,imd_decile
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,0,1,0,0,0,1,0,0,0,1,0,27,5
2,1,0,1,0,0,1,0,0,0,0,1,0,26,4
3,1,1,0,0,1,0,0,0,0,0,1,0,24,3
4,1,0,1,0,0,0,0,0,1,0,1,0,24,2
5,1,1,0,0,0,0,0,0,1,0,0,0,24,4
6,1,0,1,0,0,0,0,0,1,1,0,0,27,2
7,1,1,0,0,0,1,0,0,0,0,0,0,21,6
8,1,0,1,0,0,0,0,0,1,0,0,1,18,4
9,1,0,1,0,0,0,0,0,1,1,1,1,26,2


In [39]:
using DataFrames, Statistics

# Function to get descriptive statistics
function get_descriptive_stats(group::DataFrame, column::Symbol)
    if column == :age
        count_val = count(!ismissing, group[:, column])
    else
        count_val = sum(group[:, column] .== 1)
    end
    mean_val = mean(group[:, column])
    std_val = std(group[:, column])
    return count_val, mean_val, std_val
end

get_descriptive_stats (generic function with 1 method)

In [48]:
variables = setdiff(Symbol.(names(df)), [:w, :y])
control_stats = Dict(var => get_descriptive_stats(control, var) for var in variables)
treatment_stats = Dict(var => get_descriptive_stats(treatment, var) for var in variables)



MethodError: MethodError: no method matching getindex(::Tuple{Int64, Float64, Float64}, ::Symbol)

Closest candidates are:
  getindex(::Tuple, !Matched::Colon)
   @ Base tuple.jl:37
  getindex(::Tuple, !Matched::CartesianIndex{1})
   @ Base multidimensional.jl:882
  getindex(::Tuple, !Matched::Int64)
   @ Base tuple.jl:31
  ...


Dict{Symbol, Tuple{Int64, Float64, Float64}} with 13 entries:
  :gender_female            => (475, 0.580685, 0.493749)
  :postlaunch               => (387, 0.473105, 0.499582)
  :ethnicgrp_other          => (14, 0.0171149, 0.129779)
  :imd_decile               => (37, 3.48411, 1.48608)
  :ethnicgrp_mixed_multiple => (76, 0.0929095, 0.290483)
  :partners1                => (239, 0.292176, 0.455041)
  :ethnicgrp_black          => (76, 0.0929095, 0.290483)
  :ethnicgrp_white          => (607, 0.742054, 0.437772)
  :gender_transgender       => (1, 0.00122249, 0.0349642)
  :gender_male              => (342, 0.418093, 0.493547)
  :msm                      => (113, 0.138142, 0.34526)
  :age                      => (818, 23.0477, 3.59401)
  :ethnicgrp_asian          => (45, 0.0550122, 0.228144)